# System Identification of a Freely Falling Ball

The data in `data/falling_object.npy` provides the motion of a ball (only its position), freely falling (without friction) under the gravity of one of the solar system's planets. You are supposed to find which planet the ball is closest to. Perform this task in two different ways:

1. Implement the adjoint differentiation method in Python/JAX and use the gradients provided by this method to figure out the gravitational constant.
2. Use JAX 's built-in functions `jax.grad` or `jax.value_and_grad` to replace the adjoint differentiation method and repeat the task.
3. Do both methods converge? Which method is faster? Can you make JAX 's grad (or value_and_grad) work faster?

The data contains 1000 floats that correspond to the vertical motion of the ball. Each time step corresponds to 0.001 seconds of motion. The time interval between the initial and final time step is thus 1 second.

In [1]:
from data import fallingobject

# Load height trajectory of the object.
ut, t_eval, dt = fallingobject.load_data()

## Problem Definition

The following second-order ODE represents the system behavior.
$$
    \frac{d^2x}{dt^2} = \ddot x = g,            \tag{1}
$$
where $x$ is the height of the object and $g$ is the unknown gravitational acceleration.
An analytical solution $x(t)$ of this simple ODE follows from double integration:
$$
    v(t) = \int_{0}^{t} g d\tau = v(0) + g t,   \tag{2}
$$$$
    x(t) = \int_{0}^{t} v(\tau) d\tau = x(0) + v(0) t + \frac{1}{2} g t^2.  \tag{3}
$$
Given the measurements, we know the initial position $x(0)$, but the initial velocity $v(0)$ and $g$ are unknown.
Since Eq. (3) is linear in the unknown parameters, we can find the least-square estimate of $v(0)$ and $g$, but the solution will serve as **ground truth values** for the subsequent analyses.

In [2]:
import numpy as np
# Coefficient matrix of the unknown vector (v0, g) and the RHS of eq.
A = np.array([[t, 0.5*t*t] for t in t_eval])
b = np.array(ut - ut[0]).reshape(-1)
# Least squared estimate of the unknowns.
x, _, _, _ = np.linalg.lstsq(A, b, rcond=None)
g_best = x[1]
v0_best = x[0]
print(f"LS estimate of grav. acc.: {g_best:.5f}")
print(f"LS estimate of init. vel.: {v0_best:.5f}")

LS estimate of grav. acc.: -3.70002
LS estimate of init. vel.: 0.10185
